# prossesing the data

In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"


def load_all_txt(input_dir):
    all_text = ""

    for filename in os.listdir(input_dir):
        if filename.endswith(".txt"):
            file_path = os.path.join(input_dir, filename)
            with open(file_path, "r", encoding="utf-8") as f:
                all_text += f.read() + "\n"

    return all_text

In [2]:
input_directory = r"/kaggle/input/chess-analyse-books"
full_text = load_all_txt(input_directory)
print(len(full_text))

5556076


# load and test model

In [3]:
!pip install transformers accelerate pyarrow==14.0.2 datasets==2.18.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.0/38.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer, StoppingCriteria, StoppingCriteriaList
import torch

model_name = "Qwen/Qwen3-1.7B"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float32,
    device_map="auto"
)
model.eval()


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [5]:
class StopAtSentence(StoppingCriteria):
    def __call__(self, input_ids, scores, **kwargs):
        # Check last token
        last_token = input_ids[0][-1].item()
        # Stop if EOS or common sentence-ending token (".", "?", "!")
        if last_token == tokenizer.eos_token_id:
            return True
        decoded = tokenizer.decode([last_token], skip_special_tokens=True)
        if decoded.strip() in {".", "?", "!"}:
            return True
        return False

def chat_qwen3_direct(prompt: str, max_new_tokens: int = 256) -> str:
    messages = [{"role": "user", "content": prompt}]
    
    # Apply chat template WITHOUT thinking
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
        enable_thinking=False  # 🔥 Disable thinking for factual answers
    )
    
    inputs = tokenizer([text], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=0.5,      # Lower = more focused
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # stopping_criteria=StoppingCriteriaList([StopAtSentence()])  # Optional
        )
    
    # Extract only the new text
    input_len = inputs.input_ids.shape[1]
    response = tokenizer.decode(outputs[0][input_len:], skip_special_tokens=True)
    
    # Clean: remove any trailing incomplete sentence
    for end in [".", "?", "!"]:
        if end in response:
            response = response[:response.rfind(end)+1]
            break
    else:
        # No sentence end? Return first line or truncate
        response = response.split("\n")[0].strip()
    
    return response.strip()

## Test the model output on simple question

In [6]:
prompt = "Explain why controlling the center is important in chess in 2–3 sentences."
answer = chat_qwen3_direct(prompt)
print("💬 Answer:", answer)

💬 Answer: Controlling the center in chess is crucial because it allows for greater mobility and flexibility for both pieces, giving a player a significant advantage in developing strategies and attacking opponents. The center is where the most active pieces are, and controlling it often leads to better coordination and more efficient piece development. A strong center control can also restrict the opponent's options and limit their ability to launch effective attacks.


# fine tune the model

In [7]:
def chunk_text(text, tokenizer, max_length):
    # Use batched tokenization to avoid huge tensors
    tokens = tokenizer(
        text,
        return_tensors="pt",
        add_special_tokens=False,  # Avoid extra BOS/EOS in middle of book
        truncation=False,
        verbose=False
    )["input_ids"][0]

    chunks = []
    # Use full chunks only (avoid tiny trailing fragments)
    for i in range(0, len(tokens) - max_length + 1, max_length):
        chunk = tokens[i:i + max_length].tolist()
        chunks.append({"input_ids": chunk})
    
    # Optional: add last partial chunk if long enough
    remainder = len(tokens) % max_length
    if remainder > 64:  # only if >64 tokens
        chunks.append({"input_ids": tokens[-remainder:].tolist()})
    
    return chunks

## prepare the data for training

In [8]:
from peft import LoraConfig, get_peft_model, TaskType,PeftModel
lora_config = LoraConfig(
    r=32,                # LoRA rank
    lora_alpha=32,       # LoRA alpha
    target_modules=["q_proj", "v_proj"],  # common modules for LLaMA/Qwen
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Wrap model with LoRA
model = get_peft_model(model, lora_config)

In [9]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import Dataset

max_length = 256

data = chunk_text(full_text, tokenizer, max_length)
dataset = Dataset.from_list(data)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,                              
    pad_to_multiple_of=8                    
)

In [10]:
training_args = TrainingArguments(
    output_dir="./chess-qwen",
    run_name="chess-qwen-run-1",

    
    per_device_train_batch_size=2,    
    gradient_accumulation_steps=16,   

   
    num_train_epochs=3,
    learning_rate=2e-4,                
    fp16=True,                      
    bf16=False,                        

    
    save_steps=2000,                   
    logging_steps=200,                  
    save_total_limit=1,                 

    # 🚀 Other speedups
    optim="adamw_torch",                
    report_to="none",                  
    dataloader_num_workers=4,           
    remove_unused_columns=True,
)


# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=data_collator
)

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


## training loop

In [11]:
print('start training')
trainer.train()

print("Merging LoRA adapters into base model...")
model = model.merge_and_unload()
model.save_pretrained(
    "./chess-qwen-adapted",
    safe_serialization=True
)
tokenizer.save_pretrained("./chess-qwen-adapted")

start training


Step,Training Loss
200,2.875400
400,2.646000
600,2.572900


Merging LoRA adapters into base model...


('./chess-qwen-adapted/tokenizer_config.json',
 './chess-qwen-adapted/special_tokens_map.json',
 './chess-qwen-adapted/chat_template.jinja',
 './chess-qwen-adapted/vocab.json',
 './chess-qwen-adapted/merges.txt',
 './chess-qwen-adapted/added_tokens.json',
 './chess-qwen-adapted/tokenizer.json')

In [12]:
del trainer
del model
del data_collator
del dataset
del tokenizer

torch.cuda.empty_cache()

# test the adapted model on same simple question

In [13]:
model_name = "/kaggle/working/chess-qwen-adapted"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    padding_side="left"
)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-27): 28 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2048, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (up_proj): Linear(in_features=2048, out_features=6144, bias=False)
          (down_proj): Linear(in_features=6144, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen3RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwe

In [14]:
prompt = "Explain why controlling the center is important in chess in 2–3 sentences."
answer = chat_qwen3_direct(prompt)
print("💬 Answer:", answer)

💬 Answer: Controlling the center is crucial in chess because it allows a player to exert influence over the majority of the board, create strong pieces and pawns, and develop pieces more effectively. It also makes it easier to coordinate pieces and exert pressure on the opponent's king, which can lead to a decisive advantage.


# Quantizing the model using llama-cpp

In [15]:
!pip install sentencepiece
!pip install "llama-cpp-python>=0.2.0"
!git clone https://github.com/ggerganov/llama.cpp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 39.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp311-cp311-linux_x86_64.whl size=4503243 sha256=694c9175f0f0f24ac1d4c3d5527dc4b83881781a26ff449b03458f0993d38ffb
  Stored in directory: /root/.cache/pip/wheels/d8/5b/e5/a7d4b5765da347d314e8155197440c9995a962f8e4a5f52b23
Successfully built llama-cpp-python
Cloning into 'llama.cpp'...
remote: Enumerating objects: 72340, done.
remote: Counting objects: 100% (322/322), done.
remote: Compressing objects: 100% (246/246), done.
remote: Total 72340 (delta 213), reused 76 (delta 76), pack-reused 72018 (from 3)
Receiving objects: 100% (72340/72340), 242.25 MiB | 33.89 MiB/s, done.
Resolving del

In [16]:
!python llama.cpp/convert_hf_to_gguf.py \
    /kaggle/working/chess-qwen-adapted \
    --outfile /kaggle/working/chess-fp16.gguf \
    --outtype f16

Writing: 100%|███████████████████████████| 3.44G/3.44G [00:11<00:00, 297Mbyte/s]


In [17]:
!cd llama.cpp && mkdir -p build && cd build && cmake .. && make -j

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [18]:
!llama.cpp/build/bin/llama-quantize \
    /kaggle/working/chess-fp16.gguf \
    /kaggle/working/chess-q8_0.gguf \
    q8_0

main: build = 7438 (ef83fb860)
main: built with GNU 11.4.0 for Linux x86_64
main: quantizing '/kaggle/working/chess-fp16.gguf' to '/kaggle/working/chess-q8_0.gguf' as Q8_0
llama_model_loader: loaded meta data with 29 key-value pairs and 310 tensors from /kaggle/working/chess-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = qwen3
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                     general.sampling.top_k i32              = 20
llama_model_loader: - kv   3:                     general.sampling.top_p f32              = 0.950000
llama_model_loader: - kv   4:                      general.sampling.temp f32              = 0.600000
llama_model_loader: - kv   5:                               general.name str              = Ches

# compare quantize model and two pipelines

## define the funcs

In [19]:
position_analyze_prompt = """
You are a chess analysis engine.

All evaluations are from White's perspective.
Positive means advantage for White.

Given Stockfish candidate lines, evaluate the position Who is better and why
Rules:
- Write only 2 or 3 sentences.
- Do NOT mention move names.
- Do NOT mention numbers, evaluations, or scores.
- Use plain natural language.

Position analysis:
{analysis}
"""

def llm_position_analysis(stockfish_text: str,llm_func) -> str:
    prompt = position_analyze_prompt.format(
        analysis=stockfish_text
    )
    return llm_func(prompt)

In [20]:
final_analyze_prompt = """
You are a chess position evaluator.

All evaluations are from White’s perspective.

Based on the position before and after the move:
- Decide if the move is good or bad.
- If good, explain why it improves the position.
- If bad, explain why it weakens the position.
- If the change is small, briefly describe the position.

Rules:
- Write only 2 or 3 sentences.
- Do NOT mention move names.
- Do NOT mention numbers, evaluations, or scores.
- Use plain natural language.

Before position analysis:
{before_llm}

After position analysis:
{after_llm}

"""
def llm_move_analysis(before_llm, after_llm, eval_before, eval_after,llm_func):
    prompt = final_analyze_prompt.format(
        before_llm=before_llm,
        after_llm=after_llm,
        eval_before=eval_before,
        eval_after=eval_after,
    )
    return llm_func(prompt)

In [21]:
def full_llm_pipeline(sample,llm_func):
    # Step 1: LLM evaluates positions independently
    before_llm = llm_position_analysis(sample["before"],llm_func)
    after_llm  = llm_position_analysis(sample["after"],llm_func)

    # Step 2: LLM compares its OWN outputs + Stockfish eval
    final_eval = llm_move_analysis(
        before_llm,
        after_llm,
        sample['before']["eval"],
        sample['after']["eval"],
        llm_func
    )

    return {
        "before_llm": before_llm,
        "after_llm": after_llm,
        "final_analysis": final_eval
    }
    
def simple_method(sample,llm_func):
    prompt = f"""
Analyze the following move and position.
Rules:
- Write only 2 or 3 sentences.
- Do NOT mention move names.
- Do NOT mention numbers, evaluations, or scores.
- Use plain natural language.

{sample["before"]}
{sample["after"]}

Eval before: {sample['before']["eval"]}
Eval after: {sample['after']["eval"]}
"""
    return llm_func(prompt)


In [22]:
from llama_cpp import Llama

gguf_llm = Llama(
    model_path="/kaggle/working/chess-q8_0.gguf",
    n_ctx=4096,
    n_threads=8,
    temperature=0.35,
    top_p=0.9,
    repeat_penalty=1.15,
    verbose=False
)

def chat_gguf(prompt: str, max_tokens: int = 64) -> str:
    """
    Qwen-style GGUF chat, faithful to llama-cli usage
    """

    full_prompt = (
        "<|im_start|>user\n"
        ">> " + prompt.strip() + "\n"
        ">> <|im_end|>\n"
        ">> <|im_start|>assistant"
    )

    output = gguf_llm(
        full_prompt,
        max_tokens=max_tokens,
        stop=["<|im_end|>"],   # reverse-prompt equivalent
    )

    text = output["choices"][0]["text"].strip()

    # Safety cleanup: stop at first sentence end if needed
    for end in [".", "?", "!"]:
        if end in text:
            text = text[:text.rfind(end) + 1]
            break

    return text


llama_context: n_ctx_per_seq (4096) < n_ctx_train (40960) -- the full capacity of the model will not be utilized


## load the sample data with stockfish labels

In [23]:
samples = [
  {
    "move_type": "blunder",
    "move_number": 1,
    "before": {
      "fen": "r2qk2r/ppp2ppp/2n5/3n4/Q1B5/P1P1BP2/5P1P/R3K2R w KQkq - 0 14",
      "stockfish_analysis": "[White POV]: 86.8% cp:376 O-O-O Qf6 Rxd5 O-O Qc2 Rae8 Bb3 Qxf3 Rhd1 Qh3 a4 Qxh2 Kb2 g6 (Depth: 20)\n[White POV]: 85.4% cp:354 Rd1 Nxc3 Rxd8+ Rxd8 Qb3 Rd1+ Qxd1 Nxd1 Kxd1 Ke7 Bg5+ f6 Re1+ Kd7 Be3 Rd8 Kc1 g6 h4 Ne5 (Depth: 20)\n[White POV]: 30.8% cp:-162 Qb3 Nxe3 Bxf7+ Kf8 fxe3 Qe7 O-O-O Qxf7 Qxb7 Qe8 Qxc7 h5 Rhg1 Qxe3+ Kc2 Ne7 Rge1 Qf2+ Kc1 Re8 (Depth: 20)",
      "eval": "86.76111264579346"
    },
    "after": {
      "fen": "r2qk2r/ppp2ppp/2n5/3B4/Q7/P1P1BP2/5P1P/R3K2R b KQkq - 0 14",
      "stockfish_analysis": "[White POV]: 21.4% cp:-260 Qxd5 Qe4+ Qxe4 fxe4 O-O-O Ke2 g6 f3 Rhe8 h4 f5 exf5 gxf5 Kf2 Ne5 Bg5 Nd3+ Kf1 Rd7 Rd1 (Depth: 19)\n[White POV]: 90.3% cp:446 O-O Bxc6 Qf6 Bxb7 Rab8 Qd4 Qg6 Be4 c5 Qd2 Qg2 Ke2 f5 Bd5+ Kh8 Rhg1 (Depth: 19)\n[White POV]: 92.2% cp:493 Qd6 Bxc6+ bxc6 Qe4+ Qe6 Qxe6+ fxe6 f4 O-O-O h4 Rhf8 h5 Rf5 Ke2 Kb7 Rab1+ Ka8 c4 (Depth: 19)",
      "eval": "92.16514750369404"
    }
  },
  {
    "move_type": "normal",
    "move_number": 1,
    "before": {
      "fen": "rnbq1rk1/p1p1bppp/1p2pn2/3p4/2PP1B2/2N1PN2/PP3PPP/R2QKB1R w KQ - 0 7",
      "stockfish_analysis": "[White POV]: 54.0% cp:32 cxd5 Nxd5 Nxd5 exd5 Be2 Bb4+ Nd2 Be7 O-O c5 Nf3 Nc6 b3 a6 dxc5 bxc5 Ne5 Nxe5 Bxe5 Be6 (Depth: 21)\n[White POV]: 53.6% cp:29 Be2 dxc4 Bxc4 Bb7 O-O a6 Qe2 b5 Bd3 c5 dxc5 Bxc5 e4 Nbd7 Rad1 Nh5 Be3 Qc7 (Depth: 21)\n[White POV]: 52.6% cp:21 Rc1 c5 dxc5 bxc5 Be2 Bb7 O-O Nbd7 cxd5 Nxd5 Nxd5 exd5 Qb3 Qb6 Rfd1 Nf6 Ne5 Bd6 Qxb6 axb6 (Depth: 21)",
      "eval": "53.99148845555657"
    },
    "after": {
      "fen": "rnbq1rk1/p1p1bppp/1p2pn2/3p4/2PP1B2/2NBPN2/PP3PPP/R2QK2R b KQ - 1 7",
      "stockfish_analysis": "[White POV]: 53.7% cp:30 dxc4 Bxc4 Bb7 O-O Nbd7 Qe2 a6 a4 c5 Rad1 cxd4 exd4 Re8 Ne5 Bb4 Rfe1 Nxe5 dxe5 Nd5 Qg4 (Depth: 20)\n[White POV]: 54.9% cp:39 Ba6 cxd5 Nxd5 Nxd5 Qxd5 O-O Qb7 Rc1 Bxd3 Qxd3 Rc8 Ne5 f6 Nf3 c5 Rfd1 Nd7 (Depth: 20)\n[White POV]: 55.2% cp:42 c5 cxd5 Nxd5 Nxd5 exd5 Ne5 Ba6 dxc5 bxc5 O-O Bxd3 Qxd3 c4 Qe2 f6 Nf3 Nc6 b3 c3 Rac1 (Depth: 20)",
      "eval": "55.230790957432525"
    }
  },
  {
    "move_type": "inaccuracy",
    "move_number": 1,
    "before": {
      "fen": "rnb1kb1r/pp3ppp/1qp2p2/8/3P4/8/PPPB1PPP/R2QKBNR w KQkq - 0 8",
      "stockfish_analysis": "[White POV]: 55.5% cp:44 Nf3 Be6 a3 g6 c4 Bg7 Be2 O-O O-O Re8 Re1 Nd7 d5 cxd5 Be3 Qd8 cxd5 Bg4 d6 (Depth: 18)\n[White POV]: 54.6% cp:37 c3 Bf5 Bc4 Qxb2 Nf3 Qc2 Qe2+ Qe4 Qxe4+ Bxe4 O-O b5 Bb3 Kd7 a4 Bd6 axb5 cxb5 Rfe1 (Depth: 18)\n[White POV]: 52.6% cp:21 Bd3 Qxd4 Nf3 Qd5 O-O Bd6 Qe1+ Be6 Bb4 Bxb4 Qxb4 c5 Qa4+ Nd7 Be4 Qc4 Qxc4 Bxc4 (Depth: 18)",
      "eval": "55.477923510721475"
    },
    "after": {
      "fen": "rnb1kb1r/pp3ppp/1qp2p2/8/3P4/1P6/P1PB1PPP/R2QKBNR b KQkq - 0 8",
      "stockfish_analysis": "[White POV]: 43.4% cp:-53 Qxd4 Nf3 Qd8 Bd3 Bc5 Qe2+ Qe7 Qxe7+ Bxe7 O-O O-O Rfe1 Ba3 Ba5 b6 Bc3 Na6 Be4 Bd7 Nd2 (Depth: 19)\n[White POV]: 49.9% cp:-1 Bd6 c3 O-O Bd3 c5 d5 Re8+ Ne2 Bg4 c4 g6 f3 Bd7 Kf2 Na6 a3 Nb4 Bb1 (Depth: 19)\n[White POV]: 50.6% cp:5 Be6 Nf3 c5 Be3 cxd4 Qxd4 Bb4+ c3 Qxd4 Bxd4 Ba3 Bb5+ Nc6 Ke2 O-O-O Rhd1 Nxd4+ Nxd4 Bd6 (Depth: 19)",
      "eval": "50.62496744995104"
    }
  },
  {
    "move_type": "mistake",
    "move_number": 1,
    "before": {
      "fen": "r4r1k/pp1nQppp/1qp2p2/5b2/2BP4/1PP5/P2B1PPP/R3K1NR w KQ - 3 13",
      "stockfish_analysis": "[White POV]: 91.8% cp:482 Ne2 Rae8 Qa3 Qd8 O-O Nb6 Rfe1 Nxc4 bxc4 Bd3 Nf4 Bxc4 Qc5 b5 Rxe8 Rxe8 Qxc6 a5 Qf3 (Depth: 18)\n[White POV]: 90.8% cp:458 Qa3 Rfe8+ Ne2 Qd8 O-O Nb6 Ng3 Nxc4 bxc4 Bd3 Rfe1 Rxe1+ Bxe1 Bxc4 Qc5 b5 a4 a5 Nf5 Be6 (Depth: 18)\n[White POV]: 89.5% cp:428 Qd6 Qa5 Qg3 Qa3 Bc1 Rfe8+ Ne2 Qe7 Qf3 Be4 Qe3 Bxg2 Qxe7 Rxe7 Rg1 Be4 Ng3 Rae8 Be2 (Depth: 18)",
      "eval": "91.75866818720871"
    },
    "after": {
      "fen": "r4r1k/pp1nQppp/1qp2p2/5b2/2BP4/1PP5/P2B1PPP/2KR2NR b - - 4 13",
      "stockfish_analysis": "[White POV]: 81.5% cp:296 c5 dxc5 Qa5 Kb2 Rfe8 b4 Qa4 Bb3 Qxb3+ axb3 Rxe7 Nf3 Bg4 h3 Bxf3 gxf3 Ne5 Be3 Rae8 c4 (Depth: 18)\n[White POV]: 83.9% cp:330 Qa5 Kb2 b5 Bf1 c5 Nf3 h6 Re1 c4 Qa3 Qc7 Ka1 b4 Qxb4 cxb3 Qxb3 (Depth: 18)\n[White POV]: 85.0% cp:347 Qc7 Qe3 Rfe8 Qf4 Qa5 Kb2 Bg6 Bf1 b5 c4 b4 Nf3 Qa3+ Ka1 a5 Bc1 (Depth: 18)",
      "eval": "85.00508583359613"
    }
  },
  {
    "move_type": "normal",
    "move_number": 2,
    "before": {
      "fen": "r4r1k/2qn1ppp/2p2p2/pp3b2/3P2P1/QPP2N1P/P2B1P2/2KR1B1R b - - 0 18",
      "stockfish_analysis": "[White POV]: 76.0% cp:230 b4 Qb2 Be4 Bg2 a4 Rhe1 axb3 axb3 bxc3 Bxc3 Bd5 Bh1 Rfb8 Nd2 Bxh1 Rxh1 Nb6 Qc2 h6 Rde1 (Depth: 20)\n[White POV]: 76.0% cp:231 Be4 Bg2 b4 Qb2 a4 Rhe1 axb3 axb3 bxc3 Bxc3 Bd5 Bh1 Rfb8 Nd2 Bxh1 Rxh1 Nb6 Qc2 h6 Rhe1 (Depth: 20)\n[White POV]: 79.5% cp:271 Be6 Qb2 c5 dxc5 Nxc5 Nd4 b4 c4 Rfb8 Be3 a4 Nxe6 Nxb3+ Kb1 fxe6 Be2 a3 Qxb3 Qb7 (Depth: 20)",
      "eval": "79.49458649106352"
    },
    "after": {
      "fen": "r4r1k/2qn1ppp/2p2p2/pp6/3Pb1P1/QPP2N1P/P2B1P2/2KR1B1R w - - 1 19",
      "stockfish_analysis": "[White POV]: 75.3% cp:223 Bg2 b4 Qb2 a4 Rhe1 axb3 axb3 f5 cxb4 Nf6 Rxe4 Nxe4 Ne5 Nxf2 Rf1 f6 Rxf2 fxe5 Rxf5 Rxf5 (Depth: 21)\n[White POV]: 62.4% cp:101 Be2 c5 Rhe1 Qb7 Be3 Bxf3 Bxf3 Qxf3 dxc5 Ne5 Qb2 Qxh3 Qe2 Qxg4 Qxb5 h5 Rd6 Kg8 c6 Rfc8 (Depth: 21)\n[White POV]: 40.5% cp:-77 Rg1 Bxf3 Re1 a4 b4 Bd5 c4 bxc4 Qg3 Qb7 g5 Rae8 Be2 fxg5 Qxg5 Rg8 Bd1 Be4 Bc3 f6 (Depth: 21)",
      "eval": "75.30600904213377"
    }
  },
  {
    "move_type": "blunder",
    "move_number": 2,
    "before": {
      "fen": "r4r1k/2qn1ppp/2p2p2/p7/1p1Pb1P1/QPP2N1P/P2B1PB1/2KR3R w - - 0 20",
      "stockfish_analysis": "[White POV]: 76.0% cp:230 Qb2 a4 Rhe1 f5 gxf5 axb3 axb3 bxc3 Bxc3 Bd5 Bh1 f6 Qc2 Nb6 Rd3 Rfb8 Rde3 (Depth: 18)\n[White POV]: 35.5% cp:-119 cxb4 axb4 Qxb4 Rxa2 Rde1 c5 Qc3 Rc2+ Qxc2 Bxc2 Kxc2 cxd4+ Kb2 Qb6 Rc1 Rb8 b4 d3 Rhf1 Qb5 (Depth: 18)\n[White POV]: 13.6% cp:-369 Rhe1 Bxf3 cxb4 Bxg2 bxa5 Bf3 Qe7 Bxd1 Kxd1 Qc8 d5 cxd5 a3 (Depth: 18)",
      "eval": "75.9510916949111"
    },
    "after": {
      "fen": "r4r1k/2qn1ppp/2p2p2/p7/1P1Pb1P1/QP3N1P/P2B1PB1/2KR3R b - - 0 20",
      "stockfish_analysis": "[White POV]: 38.3% cp:-95 axb4 Qxb4 Rxa2 Rde1 c5 Qc3 Bd5 Re7 Qd6 Rhe1 cxd4 Nxd4 Bxg2 Re8 Ra8 Nb5 Qa6 Nc7 Qa3+ Kd1 (Depth: 21)\n[White POV]: 81.4% cp:295 Rfc8 bxa5 c5 Kb2 Qb7 Rhe1 Bxf3 Bxf3 Qxf3 b4 Qb7 dxc5 Nxc5 Ka1 Ne4 Be3 h6 Rd4 Rc2 Rc1 (Depth: 21)\n[White POV]: 82.7% cp:313 a4 bxa4 Rfb8 Ne1 Bxg2 Nxg2 c5 Bf4 Qc6 b5 Qe6 Ne3 cxd4 Rxd4 Rxb5 Re1 (Depth: 21)",
      "eval": "82.70696501038614"
    }
  },
  {
    "move_type": "blunder",
    "move_number": 3,
    "before": {
      "fen": "7k/2q2ppp/5p2/2n5/1Q2R1PN/1P5P/2rB1P2/3K3R b - - 0 27",
      "stockfish_analysis": "[White POV]: 95.5% cp:611 Rxd2+ Kxd2 Qd8+ Rd4 Nxb3+ Ke3 Nxd4 Qxd4 Qe7+ Kf3 h5 Kg2 g5 Nf3 hxg4 Qxg4 Kg7 Qg3 Qa7 (Depth: 20)\n[White POV]: 97.1% cp:705 h5 Re8+ Kh7 Kxc2 Qc6 Re3 Ne6+ Kb2 Qxh1 Qe4+ Qxe4 Rxe4 hxg4 hxg4 Nc5 Re3 g6 Ka3 g5 Nf3 (Depth: 20)\n[White POV]: 97.5% cp:730 Qc8 Kxc2 Nxe4+ Qc4 Nc5 Re1 h5 Nf5 Kh7 Re3 Qa8 Qxc5 hxg4 hxg4 Qa2+ Kd3 Qxb3+ Ke2 (Depth: 20)",
      "eval": "97.46672967731283"
    },
    "after": {
      "fen": "7k/2q2ppp/5p2/8/1Q2n1PN/1P5P/2rB1P2/3K3R w - - 0 28",
      "stockfish_analysis": "[White POV]: 100.0% cp:inf Qf8# (Depth: 20)\n[White POV]: 96.2% cp:648 Qxe4 Rxd2+ Kxd2 g6 Nf3 Qa5+ Ke2 Qa2+ Kf1 Qxb3 Kg2 h5 Rb1 Qa2 Rb8+ Kg7 g5 fxg5 Qe5+ f6 (Depth: 20)\n[White POV]: 94.8% cp:580 Bc3 Nxc3+ Kxc2 Nd5+ Qc4 Nb4+ Kb2 Qb7 Rc1 g6 Nxg6+ fxg6 Qc8+ Qxc8 Rxc8+ Kg7 Ka3 Nd3 f3 h5 (Depth: 20)",
      "eval": "100.0"
    }
  },
  {
    "move_type": "mistake",
    "move_number": 2,
    "before": {
      "fen": "r1b1r1k1/1p1n1ppp/p3pq2/3p4/P1PP4/3BPN2/5PPP/1R1Q1RK1 b - - 0 14",
      "stockfish_analysis": "[White POV]: 55.8% cp:47 dxc4 Bxc4 b6 d5 e5 Qb3 Rb8 Qa3 e4 Nd4 Bb7 Nc6 Bxc6 dxc6 Qxc6 Bxa6 Red8 Bb5 Qe6 Rfd1 (Depth: 19)\n[White POV]: 67.4% cp:145 Qd8 Qc2 g6 c5 Qc7 Rfc1 e5 Be2 e4 Nd2 Re6 Bg4 f5 Be2 Nf6 Qb3 Rc6 h3 f4 exf4 (Depth: 19)\n[White POV]: 68.7% cp:157 g6 cxd5 exd5 Qb3 Qd6 Rfc1 Rb8 Qc2 Nf6 Qc7 Qxc7 Rxc7 Be6 Ng5 Bd7 Rbxb7 Rxb7 Rxb7 Rc8 h3 (Depth: 19)",
      "eval": "68.67567266439018"
    },
    "after": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3pp3/P1PP4/3BPN2/5PPP/1R1Q1RK1 w - - 0 15",
      "stockfish_analysis": "[White POV]: 71.2% cp:181 Nxe5 Nxe5 dxe5 Rxe5 cxd5 Bf5 Qb3 Bxd3 Qxd3 Rd8 Rfd1 Rexd5 Qxd5 Rxd5 Rxd5 h5 Rd4 Qc6 h3 a5 (Depth: 19)\n[White POV]: 70.9% cp:178 dxe5 Nxe5 Nxe5 Rxe5 cxd5 Bf5 Qb3 Bxd3 Qxd3 Rd8 Rfd1 Rexd5 Qxd5 Rxd5 Rxd5 h5 Rd4 Qc6 Rd8+ Kh7 (Depth: 19)\n[White POV]: 56.2% cp:50 Be2 dxc4 Bxc4 exd4 Nxd4 Rd8 Qc2 Ne5 Be2 Rd6 Rfd1 Nc6 Nxc6 Rxc6 Qe4 (Depth: 19)",
      "eval": "71.19759194127936"
    }
  },
  {
    "move_type": "normal",
    "move_number": 3,
    "before": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3P4/P2Pp3/3BPN2/5PPP/1R1Q1RK1 w - - 0 16",
      "stockfish_analysis": "[White POV]: 31.3% cp:-157 Be2 exf3 Bxf3 Nf8 e4 b5 e5 Qb6 Be4 Bb7 Qd2 Rab8 axb5 axb5 f4 (Depth: 18)\n[White POV]: 30.8% cp:-162 Bxe4 Rxe4 Re1 Rb8 Qc2 Re8 Qc7 b5 axb5 axb5 Ne5 Qb6 Qc6 Qd8 Qd6 Rb6 Nc6 Qf6 Qc7 Rb7 (Depth: 18)\n[White POV]: 26.1% cp:-208 Bc4 exf3 Qxf3 Qg6 Qg3 Qc2 Qc7 Nf6 d6 Qg6 f3 Bh3 Qxf7+ Qxf7 Bxf7+ Kxf7 gxh3 Rxe3 Rxb7+ Kg8 (Depth: 18)",
      "eval": "31.324327335609823"
    },
    "after": {
      "fen": "r1b1r1k1/1p1n1ppp/p4q2/3P4/P2PB3/4PN2/5PPP/1R1Q1RK1 b - - 0 16",
      "stockfish_analysis": "[White POV]: 30.2% cp:-168 Rxe4 Re1 Rb8 Qc2 Re8 e4 b5 e5 Qg6 Qc7 Rb7 Qc6 Rb6 Qc7 bxa4 Ra1 Rb7 Qc6 Rb8 e6 (Depth: 19)\n[White POV]: 83.7% cp:327 Qd8 Bd3 Nf6 Qc2 Qxd5 Ne5 Be6 Bc4 Qe4 Qxe4 Nxe4 Bxe6 Rxe6 Rxb7 f6 Nd3 Rc6 h4 a5 Rfb1 (Depth: 19)\n[White POV]: 85.2% cp:350 Qd6 Nd2 b5 Bf3 Nb6 axb5 axb5 Re1 Bd7 e4 Nxd5 exd5 Rxe1+ Qxe1 Re8 Qf1 (Depth: 19)",
      "eval": "85.19528019683105"
    }
  }
]

## testing the model with huggingface format **on gpu**

### simple method

In [24]:
print(simple_method(samples[6],chat_qwen3_direct))

White wins with a decisive attack.


### chained method

In [25]:
import json

result = full_llm_pipeline(samples[6], chat_qwen3_direct)
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "before_llm": "White has a slight advantage with the queenside pawn majority and the potential to develop a pawn storm. The black king is confined to the queenside, limiting its mobility and control. The white pieces are also more active, with the rooks and knights posing a threat to the black king's position. The black king's lack of mobility and the white pieces' potential to exploit weaknesses make the position favorable for White.",
  "after_llm": "White is better and will win the game.",
  "final_analysis": "White's pawn majority and potential for a pawn storm give him a slight edge, while black's confined king and lack of mobility weaken his position."
}


## testing the model with gguf format **on cpu**

### simple method

In [26]:
print(simple_method(samples[6],chat_gguf))

<think>
Okay, so I need to analyze this chess position and figure out what's going on. Let me start by recalling the rules. The user wants a two or three-sentence analysis without mentioning move names, numbers, or scores.


### chained method

In [27]:
import json

result = full_llm_pipeline(samples[6], chat_gguf)
print(json.dumps(result, indent=2, ensure_ascii=False))

{
  "before_llm": "<think>\n<think>\nOkay, let's see. The position is pretty straightforward. White's king is in the center, and there's a lot of pawns on the kingside. Black's king is a bit deeper, which means he's in a bit more danger.",
  "after_llm": "<think>\n<think>\nOkay, let's see. The position is a bit complex with a lot of pieces involved. White has a queen on f8, which is a strong attacking piece. The black king is on the a-file, which is a bit vulnerable because it's surrounded by pawns.",
  "final_analysis": "<think>\n<think>\nOkay, let's see. The position is a bit tense with a lot of pieces on the board. White's queen is in the center, so it's a strong attacking piece. Black's king is on the a-file, which is a bit vulnerable because it's surrounded by pawns."
}
